# Checking Job Sets for Consistency and Quality

***

# Import Modules

In [1]:
%%capture
%load_ext autoreload
%autoreload 2

In [1]:
import sys
import os

sys.path.insert(
    0, os.path.join(
    os.environ["PROJ_irox"],
    "workflow"))

sys.path.insert(
    0, os.path.join(
    os.environ["PROJ_irox"],
    "data"))

In [1]:
###############################################################################
# Local Imports ###############################################################
from proj_data_irox import data_dir
from an_data_processing import load_df


# #############################################################################
# Python Modules # ############################################################
import subprocess
import datetime

import numpy as np
import pandas as pd

from ase.visualize import view
from ase.io import Trajectory
from ase import io


# #############################################################################
# IPython Imports #############################################################
from IPython.display import display

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Script Inputs

In [2]:
prop_name_list = [
    'bulk_system',
#     'coverage',
    'coverage_type',
    'facet',
    ]

# Read and Process Data Frame

In [3]:
df_pourbaix, df_ads, df_surf = load_df(
    from_file=True,
    root_dir=data_dir,
    data_dir=data_dir,
    file_name="df_master.pickle",
    process_df=True,
    )

df_m = df_ads
df_m.loc[df_m["coverage_type"] == "O-4_OH-0", "coverage_type"] = "o_covered"
df_m.loc[df_m["coverage_type"] == "O-2_OH-0", "coverage_type"] = "o_covered_2"
df_m.loc[df_m["coverage_type"] == "O-2_OH-2", "coverage_type"] = "h_covered"

Attempting to load df from pickle


In [4]:
def create_energetics_table(
    atoms_dir,
    sys_name_i,
    root_dir,
    ):
    """
    """
    #| - create_energetics_table
    os.chdir(
        os.path.join(
            atoms_dir,
            sys_name_i,
            ),
        )

    bash_comm = os.path.join(
        os.environ["PROJ_irox"],
        "scripts",
        "dft_scripts",
        "get_energies_top_sites.py",
        )

    result = subprocess.run(
        [bash_comm],
        stdout=subprocess.PIPE,
        )

    out_i = result.stdout.decode("utf-8")

    with open("energetics_summary.txt", "w") as text_file:
        text_file.write(out_i)
#     out_i = [i for i in out_i.split("\n") if i != '']

    os.chdir(root_dir)
    #__|

# Creating Traj Objects for Each 'Job Set'

In [5]:
d = datetime.datetime.today()
date_i = str(d.year) + str(d.month) + str(d.day)
print(date_i)

202013


In [6]:
root_dir = os.getcwd()

# atoms_dir = "00_job_sets_atoms_181116_02"

num_atoms_dirs = len([i for i in os.listdir(".") if "job_sets_atoms" in i])
# atoms_dir = "00_job_sets_atoms/" + str(num_atoms_dirs).zfill(2) + "_job_sets_atoms"
atoms_dir = "out_data/00_job_sets_atoms_" + date_i + "/" + str(num_atoms_dirs).zfill(2) + "_job_sets_atoms"

if not os.path.exists(atoms_dir):
    os.makedirs(atoms_dir)

groups = df_m.groupby(by=prop_name_list)
for i_ind, (props_i, group_i) in enumerate(groups):
    # images_list = [i for i in group_i["atoms_object"].tolist() if i is not None]
    # final_atoms_list = [i[-1] for i in images_list]

    sys_name_i = str(i_ind).zfill(2) + "_" + "___".join(props_i)

    sys_i_dir = os.path.join(
        atoms_dir,
        sys_name_i,
        )

    if not os.path.exists(sys_i_dir):
        os.makedirs(sys_i_dir)

    for j_cnt, row_j in group_i.iterrows():

        name_j = row_j["adsorbate"]

        if not pd.isna(row_j["ooh_direction"]):
            name_j += "_" + row_j["ooh_direction"]


        # name_j += ".traj"
        name_j += ".json"

        if row_j["atoms_object"] is None:
            continue
        else:
            atoms_j = row_j["atoms_object"][-1]

            atoms_j_full_path = os.path.join(
                atoms_dir,
                sys_name_i,
                name_j,
                )

            print(atoms_j_full_path)

            # io.write(
            #     atoms_j_full_path,
            #     atoms_j,
            #     )


    # create_energetics_table(
    #     atoms_dir,
    #     sys_name_i,
    #     root_dir,
    #     )

out_data/00_job_sets_atoms_202013/00_job_sets_atoms/00_IrO2___h_covered___100/bare.json
out_data/00_job_sets_atoms_202013/00_job_sets_atoms/00_IrO2___h_covered___100/ooh_up.json
out_data/00_job_sets_atoms_202013/00_job_sets_atoms/00_IrO2___h_covered___100/o.json
out_data/00_job_sets_atoms_202013/00_job_sets_atoms/00_IrO2___h_covered___100/oh.json
out_data/00_job_sets_atoms_202013/00_job_sets_atoms/01_IrO2___h_covered___110/bare.json
out_data/00_job_sets_atoms_202013/00_job_sets_atoms/01_IrO2___h_covered___110/ooh_up.json
out_data/00_job_sets_atoms_202013/00_job_sets_atoms/01_IrO2___h_covered___110/o.json
out_data/00_job_sets_atoms_202013/00_job_sets_atoms/01_IrO2___h_covered___110/oh.json
out_data/00_job_sets_atoms_202013/00_job_sets_atoms/02_IrO2___o_covered___100/bare.json
out_data/00_job_sets_atoms_202013/00_job_sets_atoms/02_IrO2___o_covered___100/ooh_up.json
out_data/00_job_sets_atoms_202013/00_job_sets_atoms/02_IrO2___o_covered___100/o.json
out_data/00_job_sets_atoms_202013/00_jo